## This notebook is part of self learning Quantum Computing Algorithms

## Description: This program optimizes the stock portfolio
### With real time data

In [10]:
import numpy as np
from qiskit_finance.data_providers import RandomDataProvider
from pandas_datareader import data
import datetime
import pandas as pd

In [11]:
num_assets = 4
seed = 123
start=datetime.datetime(2021,6,1)
end=datetime.datetime(2021,6,30)
# Generate expected return and covariance matrix from (random) time-series
stocks = ['AAPL', 'NKE', 'GOOGL', 'AMZN']

In [12]:
# Import data
df = data.DataReader(stocks, 'yahoo', start=start, end=end)

In [13]:
df.head()

Attributes   Adj Close                                             Close  \
Symbols           AAPL         NKE        GOOGL         AMZN        AAPL   
Date                                                                       
2021-06-01  124.094070  134.509995  2381.179932  3218.649902  124.279999   
2021-06-02  124.872902  134.169998  2370.590088  3233.989990  125.059998   
2021-06-03  123.355186  134.169998  2347.580078  3187.010010  123.540001   
2021-06-04  125.701668  133.740005  2393.570068  3206.219971  125.889999   
2021-06-07  125.711655  133.949997  2402.300049  3198.010010  125.900002   

Attributes                                              High              ...  \
Symbols            NKE        GOOGL         AMZN        AAPL         NKE  ...   
Date                                                                      ...   
2021-06-01  134.509995  2381.179932  3218.649902  125.349998  138.050003  ...   
2021-06-02  134.169998  2370.590088  3233.989990  125.239998  135.490005  ...   
2021-06-03  134.169998  2347.580078  3187.010010  124.849998  134.690002  ...   
2021-06-04  133.740005  2393.570068  3206.219971  126.160004  134.940002  ...   
2021-06-07  133.949997  2402.300049  3198.010010  126.320000  135.199997  ...   

Attributes          Low                     Open                           \
Symbols           GOOGL         AMZN        AAPL         NKE        GOOGL   
Date                                                                        
2021-06-01  2354.280029  3209.060059  125.080002  137.850006  2374.439941   
2021-06-02  2352.360107  3208.000000  124.279999  134.589996  2389.149902   
2021-06-03  2329.520020  3184.030029  124.680000  133.199997  2345.729980   
2021-06-04  2362.409912  3198.810059  124.070000  134.500000  2369.270020   
2021-06-07  2381.000000  3172.199951  126.169998  134.000000  2389.439941   

Attributes                 Volume                             
Symbols            AMZN      AAPL      NKE    GOOGL     AMZN  
Date                                                          
2021-06-01  3243.500000  67637100  5577900  1167700  2430000  
2021-06-02  3223.100098  59278900  5226100  1057900  2014500  
2021-06-03  3204.229980  76229200  5027400   934800  2398300  
2021-06-04  3212.000000  75169300  5217100  1222900  2249700  
2021-06-07  3197.330078  71057600  3765000  1206000  2215800  

[5 rows x 24 columns]

In [14]:
df = df['Adj Close']

In [15]:
df.head()

Symbols,AAPL,NKE,GOOGL,AMZN
Date,,,,
2021-06-01,124.094070,134.509995,2381.179932,3218.649902
2021-06-02,124.872902,134.169998,2370.590088,3233.989990
2021-06-03,123.355186,134.169998,2347.580078,3187.010010
2021-06-04,125.701668,133.740005,2393.570068,3206.219971
2021-06-07,125.711655,133.949997,2402.300049,3198.010010


In [20]:
df_data = []
for stock in stocks:
    df_data.append(df[stock].values.tolist())
print(df_data)

[[124.09407043457031, 124.8729019165039, 123.35518646240234, 125.70166778564453, 125.71165466308594, 126.5503921508789, 126.93981170654297, 125.92134094238281, 127.15947723388672, 130.2847900390625, 129.44606018066406, 129.95529174804688, 131.59283447265625, 130.26483154296875, 132.10208129882812, 133.77955627441406, 133.49998474121094, 133.21041870117188, 132.91087341308594, 134.578369140625, 136.1260528564453, 136.75511169433594], [134.50999450683594, 134.1699981689453, 134.1699981689453, 133.74000549316406, 133.9499969482422, 133.35000610351562, 131.83999633789062, 130.97999572753906, 131.94000244140625, 131.36000061035156, 130.2899932861328, 130.39999389648438, 128.9199981689453, 128.41000366210938, 130.0800018310547, 132.47999572753906, 133.10000610351562, 133.60000610351562, 154.35000610351562, 152.36000061035156, 155.9499969482422, 154.49000549316406], [2381.179931640625, 2370.590087890625, 2347.580078125, 2393.570068359375, 2402.300048828125, 2398.43994140625, 2407.93994140625,

## $\min\limits_{x \in \{0, 1\}^{n}} q \ x^{T}\Sigma x − \mu^{T}x \\ \text{or} \\ \max\limits_{x \in \{0, 1\}^{n}} \mu^{T}x - q \ x^{T}\Sigma x \\ \text{subject to}: 1^{T} x = B$

##### $\text{ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html} \\  x \in \{0, 1\}^{n} \ \text{denotes the vector of binary decision variables, which indicate which assets to pick} \left(x[i]=1\right) \text{and which not to pick} \left(x[i]=0\right), \\ \mu \in R^{n} \ \text{defines the expected returns for the assets}, \\ \Sigma \in R^{n \times n} \text{specifies the covariances between the assets}, \\ q \gt 0 \ \text{controls the risk appetite of the decision maker}, \\ \text{and B denotes the budget, i.e. the number of assets to be selected out of n.}$

## $\text{The equality constraint} \ 1^{T}x = B \ \text{is mapped to a penalty term} \ \left(1^{T} x − B\right)^{2}$

## $\text{Assumptions}:\\ \ \cdot \text{all assets have the same price} \left(\text{normalized to 1}\right) \\ \ \cdot \text{the full budget B has to be spent, i.e., one has to select exactly B assets.}$

In [22]:
def divide_2(val_1, val_2):
    if val_2 == 0:
        if val_1 == 0:
            return 1
        return np.nan
    return val_1 / val_2

## Calculate mean returns 
## $\mu \in R^{n} \ \text{defines the expected returns for the assets}$

In [5]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(data)[:, 1:], np.array(data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## Calculate mean covariance
##  $\Sigma \in R^{n \times n} \text{specifies the covariances between the assets}$

In [25]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
sigma = np.cov(period_returns)

## plot sigma

In [27]:
import matplotlib.pyplot as plt

In [23]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## $q \gt 0 \ \text{controls the risk appetite of the decision maker}$

In [24]:
q = 0.5 # set risk factor

## $\text{B denotes the budget, i.e. the number of assets to be selected out of n.}$

In [26]:
budget = 2 # set budget (B)

### ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html

In [28]:
penalty = num_assets # set parameter to scale the budget penalty term

## Model

In [29]:
from docplex.mp.model import Model

In [30]:
mdl = Model('portfolio model')
#x = mdl.binary_var_list(num_assets)

In [31]:
x = list()
for i in range(num_assets):
    x.append(mdl.binary_var(name="x_{0}".format(i)))
print(x)

[docplex.mp.Var(type=B,name='x_0'), docplex.mp.Var(type=B,name='x_1'), docplex.mp.Var(type=B,name='x_2'), docplex.mp.Var(type=B,name='x_3')]


## $\max\limits_{x \in \{0, 1\}}  \mu^{T} * x - q * x^{T} * \Sigma * x \\ \text{linear} = c^{T}x, \text{qudratic} = x^{T}Qx = \sum\limits_{i, j = 1}^{n} x_{i}^{T} \Sigma_{ij} x_{j} \\ \text{objective} = \text{linear - }\left(\text{risk_factor * qudratic}\right)$

In [32]:
linear = np.dot(mu, x) # mu^T * x
qudratic = mdl.sum([x[i]*sigma[i][j]*x[j] for i in range(num_assets) for j in range(num_assets)])
objective = linear - q * qudratic
mdl.maximize(objective)

## budget constraint:  $\sum\limits_{i =1}^{n} 1^{T}x_{i} == \text{budget}$

In [33]:
cost = mdl.sum([x[i] for i in range(num_assets)])
mdl.add_constraint(cost == budget, ctname='budget')

docplex.mp.LinearConstraint[budget](x_0+x_1+x_2+x_3,EQ,2)

## converting to Quadratic Program

## removing the constraint to create the QUBO

In [35]:
from qiskit_optimization.translators import from_docplex_mp

In [36]:
mod = from_docplex_mp(mdl)

In [37]:
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio model

Maximize
 obj: 0.004682261762 x_0 + 0.007159640330 x_1 + 0.001227757962 x_2
      + 0.003234622966 x_3 + [ - 0.000095526386 x_0^2 + 0.000075990183 x_0*x_1
      - 0.000091374658 x_0*x_2 - 0.000085351262 x_0*x_3 - 0.001244762113 x_1^2
      + 0.000012725679 x_1*x_2 + 0.000326101317 x_1*x_3 - 0.000063470332 x_2^2
      - 0.000047494852 x_2*x_3 - 0.000126959488 x_3^2 ]/2
Subject To
 budget: x_0 + x_1 + x_2 + x_3 = 2

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_0 x_1 x_2 x_3
End



## QAOA

In [38]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit import Aer

In [39]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

In [40]:
def print_result(result):
    selection = result.x
    res = list()
    for i in range(len(selection)):
        if selection[i]:
            res.append(stocks[i])
    value = result.fval
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))
    print('Optimal: selected stocks {}'.format(res))

    eigenstate = result.min_eigen_solver_result.eigenstate
    eigenvector = eigenstate if isinstance(eigenstate, np.ndarray) else eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = QuadraticProgramToQubo().convert(mod).objective.evaluate(x)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [41]:
algorithm_globals.random_seed = 1234
backend = Aer.get_backend('statevector_simulator')
cobyla = COBYLA()
cobyla.set_options(maxiter=250)
quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)
qaoa_mes = QAOA(optimizer=cobyla, reps=3, quantum_instance=quantum_instance)
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa_mes)
result = eigen_optimizer.solve(mod)
print_result(result)

Optimal: selection [1. 1. 0. 0.], value 0.0112
Optimal: selected stocks ['AAPL', 'NKE']

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 0 1 1]	-0.0043		0.1682
 [1 0 1 0]	-0.0058		0.1675
 [0 1 1 0]	-0.0077		0.1667
 [1 0 0 1]	-0.0078		0.1667
 [0 1 0 1]	-0.0099		0.1656
 [1 1 0 0]	-0.0112		0.1650
 [1 1 1 1]	4.0539		0.0001
 [0 0 0 0]	4.0696		0.0001
 [1 1 0 1]	1.0029		0.0000
 [1 1 1 0]	1.0050		0.0000
 [0 1 1 1]	1.0063		0.0000
 [0 0 1 0]	1.0162		0.0000
 [0 0 0 1]	1.0142		0.0000
 [1 0 1 1]	1.0085		0.0000
 [1 0 0 0]	1.0128		0.0000
 [0 1 0 0]	1.0109		0.0000


## If you have any queries on this notebook please reach to me bala.na@hcl.com